### Решение соревнования https://zindi.africa/competitions/classification-for-landslide-detection демонстрирует процесс создания и обучения модели глубокого обучения для бинарной классификации оползней на основе мультиспектральных данных и Sentinel-2.
### Метрика соревнования - f1
### После импорта из kaggle notebook некоторые выводы после запуска кода пропали

### Выполняем установку библиотеки comet_ml. Comet ML – это платформа для отслеживания, сравнения и оптимизации экспериментов машинного обучения

In [ ]:
# Import necessary libraries
!pip install comet_ml > /dev/null 2>&1

### Импорт необходимых библиотек

In [ ]:
import comet_ml
COMET_API_KEY = "My CometML"
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Input, GlobalAveragePooling2D
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
from scipy.ndimage import rotate, shift, zoom
data = "/kaggle/input/slideandseekclasificationlandslidedetectiondataset"

### Определение путей к данным и предварительная загрузка CSV-файлов train и test

In [ ]:
train_csv_path = f'{data}/Train.csv'
test_csv_path = f'{data}/Test.csv'
train_data_path = f'{data}/train_data/train_data'
test_data_path = f'{data}/test_data/test_data'

train_df = pd.read_csv(train_csv_path)
print("Train.csv:")
print(train_df.head())

### Визуализация распределения классов (Нет последствий оползня/ после оползня)

In [ ]:
label_counts = train_df['label'].value_counts()
labels = ['No Landslide', 'Landslide']

plt.figure(figsize=(6, 4))
plt.bar(labels, label_counts.values, color=['skyblue', 'salmon'])
plt.xlabel("Class Label")
plt.ylabel("Frequency")
plt.title("Distribution of Labels in Training Set")
plt.show()

### Как видно из графика есть сильный дисбаланс классов No Landslide ~ 6000 изображений, Landslide ~ 1400 изображений
### Далее эта проблема будет решена с помощью Focal loss

### Функция загрузки и нормализации изображений, а также их визуализация
###  Изображения состоят из 12 каналов (4 оптических из Sentinel-2 и 8 SAR из Sentinel-1) и размером 64x64. Для каждого канала выполняется надежная нормализация по процентилям (2-й и 98-й), чтобы сделать значения пикселей в диапазоне [0, 1] и уменьшить влияние выбросов, которые могут быть в спутниковых данных. Также были протестированы нормализация по 1/99 процентилям и minmax шкалирование(закоментированный код), но они дали результаты похуже.

In [ ]:
def load_and_normalize_npy_image(image_id, folder_path):
    image_path = os.path.join(folder_path, f"{image_id}.npy")
    
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image not found: {image_path}")
    
    img = np.load(image_path).astype(np.float32)

    normalized_img = img
    for band in range(img.shape[2]):
        band_data = img[:, :, band]
        
        p2, p98 = np.percentile(band_data, (2, 98))
        normalized_img[:, :, band] = np.clip((band_data - p2) / (p98 - p2 + 1e-6), 0, 1)
        
    #     p1, p99 = np.percentile(band_data, (1, 99))

    #     clipped_data = np.clip(band_data, p1, p99)
    #     mean_val = np.mean(clipped_data)
    #     std_val = np.std(clipped_data)
        
    #     if std_val > 0:
    #         normalized_img[:, :, band] = (clipped_data - mean_val) / std_val
    #     else:
    #         normalized_img[:, :, band] = clipped_data - mean_val
    
    # for band in range(img.shape[2]):
    #     band_data = normalized_img[:, :, band]
    #     min_val, max_val = np.min(band_data), np.max(band_data)
    #     if max_val > min_val:
    #         normalized_img[:, :, band] = (band_data - min_val) / (max_val - min_val)
    
    return normalized_img

band_descriptions = [
    "Red", "Green", "Blue", "Near Infrared",
    "Descending VV (Vertical-Vertical)", "Descending VH (Vertical-Horizontal)",
    "Descending Diff VV", "Descending Diff VH",
    "Ascending VV (Vertical-Vertical)", "Ascending VH (Vertical-Horizontal)",
    "Ascending Diff VV", "Ascending Diff VH"
]

example_ids = train_df['ID'].sample(2,random_state = 42).values

for image_id in example_ids:
    img_normalized = load_and_normalize_npy_image(image_id, train_data_path)

    fig, axes = plt.subplots(3, 4, figsize=(20, 12))
    fig.suptitle(f"Sample Image ID: {image_id} - All 12 Bands", fontsize=16)

    for band in range(12):
        row = band // 4
        col = band % 4
        axes[row, col].imshow(img_normalized[:, :, band], cmap='gray')
        axes[row, col].set_title(f"Band {band + 1}: {band_descriptions[band]}")
        axes[row, col].axis('off')

    plt.subplots_adjust(wspace=0.3, hspace=0.4)
    plt.show()

### Вот так выглядят два 12 канальных изображения (первые три канала привычные RGB, дальше инфракрсаное и другие виды производимых и принимаемых Sentinel 2 излучений.

### Определение генератора данных для загрузки данных батчами с рандомными примерами из train и применение слуяайных аугментаций ( поворот 15 градусов, сдвиги по ширине и высоте, масштабирование, отражения и заполнение появившихся пропусков у краев.)

In [ ]:
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

class LandslideDataGenerator(Sequence):
    def __init__(self, df, data_path, batch_size=32, augment=False, shuffle=True):
        self.df = df.reset_index(drop=True)
        self.data_path = data_path
        self.batch_size = batch_size
        self.augment = augment
        self.shuffle = shuffle
        self.on_epoch_end()

        if self.augment:
            self.augmentor = ImageDataGenerator(
                rotation_range=15,
                width_shift_range=0.1,
                height_shift_range=0.1,
                shear_range=0.1,
                zoom_range=0.1,
                horizontal_flip=True,
                vertical_flip=True,
                fill_mode='reflect' 
            )
        else:
            self.augmentor = None

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle:
            np.random.shuffle(self.indexes)
        
    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_df = self.df.iloc[batch_indexes]
        
        batch_X = []
        batch_y = []
        
        for _, row in batch_df.iterrows():
            image_id = row['ID']
            label = row['label']
           
            image = load_and_normalize_npy_image(image_id, self.data_path)
            batch_X.append(image)
            batch_y.append(label)
        
        batch_X = np.array(batch_X, dtype=np.float32)
        batch_y = np.array(batch_y, dtype=np.float32).reshape(-1, 1)

        if self.augment and self.augmentor:
            batch_X = next(self.augmentor.flow(batch_X, batch_size=len(batch_X), shuffle=False))
        return batch_X, batch_y


### Проверка работы LandslideDataGenerator

In [ ]:
folder_path = '/kaggle/input/slideandseekclasificationlandslidedetectiondataset/train_data/train_data/'

batch_size = 16

train_df_split, val_df_split = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['label'])

train_gen = LandslideDataGenerator(train_df_split, folder_path, batch_size=batch_size, augment=True, shuffle=True)
val_gen = LandslideDataGenerator(val_df_split, folder_path, batch_size=batch_size, augment=False, shuffle=False)

X_batch, y_batch = train_gen[0]

X_batch.shape, y_batch.shape

### Как видим в X батче 16 изображений 64x64 по 12 каналов и в label(y) батче 16 соответствующих изображениям классов

### Вывод y_batch

In [ ]:
y_batch

### Определение функции потерь Focal Loss
### Focal Loss был разработан для решения проблемы дисбаланса классов в задачах, где один класс значительно преобладает над другим (как в случае с оползнями). Он модифицирует стандартную кросс-энтропийную потерю, уменьшая вес "легких" (хорошо классифицируемых) примеров и увеличивая вес "сложных" (плохо классифицируемых) примеров, особенно для миноритарного класса.

### gamma (параметр фокусировки): контролирует, насколько сильно подавляются "легкие" примеры. Большее gamma -> сильнее фокусировка на сложных.

### alpha (балансирующий фактор):Для редкого положительного класса alpha обычно устанавливается высоким (например, 0.75 или 0.9 в зависимости от степени дисбаланса). В текущей реализации alpha=None по умолчанию, что означает автоматический расчет весов(по количеству примеров классов в train).

In [ ]:
def focal_loss(gamma=2.0, alpha=0.75):
    """
    Focal Loss for binary classification.

    Parameters:
        gamma (float): Focusing parameter; typically set to 2.0.
        alpha (float): Balancing factor; typically set to 0.25.

    Returns:
        Binary Focal Loss function.
    """
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())

        if alpha is None:

            pos_weight = K.mean(1 - y_true)
            neg_weight = K.mean(y_true)
            alpha_t = tf.where(K.equal(y_true, 1), pos_weight, neg_weight)
        else:
            alpha_t = tf.where(K.equal(y_true, 1), alpha, 1 - alpha)

        p_t = tf.where(K.equal(y_true, 1), y_pred, 1 - y_pred)

        fl = -alpha_t * K.pow(1 - p_t, gamma) * K.log(p_t)
        return K.mean(fl)
    
    return focal_loss_fixed

### Стандартные метрики Precision , Recall  и F1-score

In [ ]:
def precision_m(y_true, y_pred):
    y_pred_bin = tf.cast(y_pred >= 0.5, tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    true_positives = tf.reduce_sum(y_true * y_pred_bin)
    predicted_positives = tf.reduce_sum(y_pred_bin)
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall_m(y_true, y_pred):
    y_pred_bin = tf.cast(y_pred >= 0.5, tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    true_positives = tf.reduce_sum(y_true * y_pred_bin)
    possible_positives = tf.reduce_sum(y_true)
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

### Далее были проведены более 50 эксперементов над различными архитектурами: простые CNN модели разной глубины, с разными функциями активаций (RelU LeakyReLU, ELU) с разными параметрами для dropout и размероми свертки, с разными гиперпараметрами lr, batch_size и т.д.
### График f1 на валидационной выборке всех экспериментов с Cnn:
![all_cnn](https://github.com/MakarRybkin/Slide_and_seek_classification/raw/master/images_for_report/all_cnn_f1.png)
### При таком подходе удалось достичь f1 =0.8054
### График f1 на валидационной выборке лучшего из экспериментов с Cnn:
![best_cnn.png](https://github.com/MakarRybkin/Slide_and_seek_classification/raw/master/images_for_report/best_cnn.png)
### Пример одной из моделей CNN можно увидеть в закомментированном коде ниже

In [ ]:
# model = Sequential([
#
#     Input(shape=X_batch.shape[1:]),
#     Conv2D(32, (3, 3), activation='relu', padding='same'),
#     BatchNormalization(),
#     Conv2D(32, (3, 3), activation='relu', padding='same'),
#     MaxPooling2D((2, 2)),
#     Dropout(0.1),
    
#
#     Conv2D(64, (3, 3), activation='relu', padding='same'),
#     BatchNormalization(),
#     Conv2D(64, (3, 3), activation='relu', padding='same'),
#     MaxPooling2D((2, 2)),
#     Dropout(0.1),
    
#
#     Conv2D(128, (3, 3), activation='relu', padding='same'),
#     BatchNormalization(),
#     Conv2D(128, (3, 3), activation='relu', padding='same'),
#     MaxPooling2D((2, 2)),
#     Dropout(0.15),
    
#
#     Conv2D(256, (3, 3), activation='relu', padding='same'),
#     BatchNormalization(),
#     Conv2D(256, (3, 3), activation='relu', padding='same'),
#     GlobalAveragePooling2D(),  # Better than Flatten for reducing parameters
#     Dropout(0.15),
    
#
#     Dense(64, activation='relu'),
#     BatchNormalization(),
#     Dropout(0.3),
#     Dense(1, activation='sigmoid')
# ])

# optimizer = AdamW(
#     learning_rate=0.005,  
#     weight_decay=0.01,
#     beta_1=0.9,
#     beta_2=0.999
# )

# model.compile(
#         optimizer=optimizer,
#         loss=focal_loss(gamma=2.0, alpha=None),  # Auto-balanced
#         metrics=[f1_m, precision_m, recall_m]
#     )


### После этого были проведены эксперементы также с разными параметрами и гиперпараметрами для более сложных готовых моделей (EfficientNet(B0 - B7) EfficientNetV2L, разной глубины ResNet и DenseNet после которых шла голова ( head) модели, состоящая из  GlobalAveragePooling2D() для уменьшения  размерности после выхода и несколько слоев FCNN

### График f1 на валидационной выборке всех экспериментов с этими архитектурами:

![all_ef_res_dense.png](https://github.com/MakarRybkin/Slide_and_seek_classification/raw/master/images_for_report/all_ef_res_dense.png)
### В результате лучшие модели оказались EfficientNet B4 (лучший f1 ~ 0.8389), ResNet50 (лучший f1 ~ 0.838) DenseNet_169 (лучший f1 ~ 0.841)

### График f1 на валидационной выборке для лучшего эксперимента с EfficientNetB4 :
![best_eff_b4.png](https://github.com/MakarRybkin/Slide_and_seek_classification/raw/master/images_for_report/best_eff_b4.png)

### График f1 на валидационной выборке для лучшего эксперимента с ResNet50 :
![best_resnet.png](https://github.com/MakarRybkin/Slide_and_seek_classification/raw/master/images_for_report/best_resnet.png)

### График f1 на валидационной выборке для лучшего эксперимента с DenseNet169:
![best_densenet.png](https://github.com/MakarRybkin/Slide_and_seek_classification/raw/master/images_for_report/best_densenet.png)
### Закомментированный код для обучения одной из моделей из этой серии можно увидеть ниже

In [ ]:
# from tensorflow.keras.applications import EfficientNetB3 , EfficientNetV2L , DenseNet169, ResNet50
# from tensorflow.keras.layers import LeakyReLU, ELU , ReLU
# input_shape = X_batch.shape[1:] 

# base_efficientnet = EfficientNetB3(weights=None, include_top=False, input_shape=input_shape)

# # densnet_169 = DenseNet169(weights=None, include_top=False, input_shape=input_shape)

# # resnet50 = ResNet50(weights=None, include_top=False, input_shape=input_shape)

# model = Sequential([
#     base_efficientnet,
#     GlobalAveragePooling2D(),
    
#     Dense(512), 
#     BatchNormalization(),
#     LeakyReLU(alpha = 0.01), 
#     Dropout(0.3),
    
#     Dense(256),
#     BatchNormalization(),
#     LeakyReLU(alpha = 0.01),
#     Dropout(0.2),
    
#     Dense(64),
#     BatchNormalization(),
#     LeakyReLU(alpha = 0.01),
#     Dropout(0.2),
#     Dense(1, activation='sigmoid')
# ])


# optimizer_efficientnet = tf.keras.optimizers.AdamW(
#     learning_rate=0.001,
#     weight_decay=0.01,
#     beta_1=0.9,
#     beta_2=0.999
# )

# model.compile(
#     optimizer=optimizer_efficientnet,
#     loss=focal_loss(gamma=2.0, alpha=None), 
#     metrics=[f1_m, precision_m, recall_m] 
# )



### Определение констант и гиперпараметров

### В результате прошлых экспериментов были выявлены лучшие гиперпараметры и архитектуры (из исследуемых) они будут использоваться ниже для объединения в ансамбль

In [ ]:
from tensorflow.keras.applications import EfficientNetB5 , EfficientNetV2L , DenseNet169, ResNet50
from tensorflow.keras.layers import LeakyReLU, ELU , ReLU
MODEL_TYPES_FOR_ENSEMBLE = [
    'EfficientNetB5',
    'DenseNet169',
    'ResNet50'
]
NUM_RUNS_PER_MODEL_TYPE = 2
BASE_RANDOM_SEED = 100


LEARNING_RATE = 0.001
BATCH_SIZE_PHASE1 = 16
EPOCHS_PHASE1 = 30
BATCH_SIZE_PHASE2 = 32
EPOCHS_PHASE2_MAX = 70
PATIENCE_EARLY_STOPPING = 25
PATIENCE_REDUCE_LR = 6
MIN_LR = 1e-8
DROPOUT_RATE_DENSE1 = 0.2
DROPOUT_RATE_DENSE2 = 0.15
DROPOUT_RATE_DENSE3 = 0.15

COMET_PROJECT_NAME = "landslide-ensemble-custom-loop"

### Функция создания и компиляции одной модели (будет использоваться ниже для каждой модели в ансамбле). В зависимости от model_type (EfficientNetB5, DenseNet169 или ResNet50), загружается соответствующая архитектура из tensorflow.keras.applications. weights=None указывает, что модели загружаются без предварительно обученных весов ImageNet. оптимизатор и компилятор для модели.

### К базовой модели добавляются следующие слои для классификации:
#### GlobalAveragePooling2D(): Уменьшает размерность пространственных признаков до одного вектора, усредняя значения по ширине и высоте.

#### Dense (полносвязные слои): Два полносвязных слоя (256 и 64 нейрона) с функциями активации LeakyReLU для нелинейности.

#### BatchNormalization(): Нормализует активации для ускорения обучения и повышения стабильности.

#### Dropout(): Применяет отсечение нейронов для предотвращения переобучения.

#### Финальный Dense(1, activation='sigmoid'): Выходной слой с одним нейроном и sigmoid-активацией для бинарной классификации (выход в диапазоне [0, 1], интерпретируемый как вероятность наличия оползня).

#### Используется tf.keras.optimizers.AdamW. Это вариант Adam-оптимизатора с явным учетом весового распада (weight decay), что помогает предотвратить переобучение.

### Компиляция модели: Модель компилируется с:

#### optimizer: Определенным выше AdamW.

#### loss: Пользовательской функцией focal_loss для работы с дисбалансом классов.

#### metrics: Пользовательскими метриками f1_m, precision_m, recall_m для оценки производительности.

In [ ]:
def create_compiled_model(model_type, input_shape):
    base_model = None
    if model_type == 'EfficientNetB5':
        base_model = EfficientNetB5(weights=None, include_top=False, input_shape=input_shape)
    elif model_type == 'DenseNet169':
        base_model = DenseNet169(weights=None, include_top=False, input_shape=input_shape)
    elif model_type == 'ResNet50':
        base_model = ResNet50(weights=None, include_top=False, input_shape=input_shape)

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),

        Dense(256),
        BatchNormalization(),
        LeakyReLU(alpha=0.01), 
        Dropout(DROPOUT_RATE_DENSE2),
        
        Dense(64),
        BatchNormalization(),
        LeakyReLU(alpha=0.01), 
        Dropout(DROPOUT_RATE_DENSE3),
        Dense(1, activation='sigmoid')
    ])

    optimizer = tf.keras.optimizers.AdamW(
        learning_rate=LEARNING_RATE,
        weight_decay=0.01,
        beta_1=0.9,
        beta_2=0.999
    )

    model.compile(
        optimizer=optimizer,
        loss=focal_loss(gamma=2.0, alpha=None),
        metrics=[f1_m, precision_m, recall_m]
    )
    return model

###  Основной цикл обучения ансамбля моделей

### Инициализация: Определяются переменные для хранения путей к обученным моделям и метаданных.

#### Двойной цикл: Код проходит по каждому model_type (EfficientNetB5, DenseNet169, ResNet50) и для каждого типа выполняет NUM_RUNS_PER_MODEL_TYPE (в данном случае 2) независимых тренировок. Это создает разнообразие в ансамбле.

#### Воспроизводимость: На каждом запуске устанавливается новый current_seed для tf.random, np.random и os.environ['PYTHONHASHSEED'], чтобы обеспечить уникальность случайной инициализации и операций, но при этом сохранить возможность воспроизвести каждый отдельный запуск, зная его сид.

#### Интеграция с Comet ML:

#### Перед началом каждого нового эксперимента создается новый объект comet_ml.Experiment.

#### Устанавливается имя эксперимента, которое будет отображаться в панели Comet ML.

#### Это позволяет отслеживать метрики, потери и другие данные для каждого запуска модели отдельно.

#### Двухфазное обучение (Two-Phase Training): Каждая модель обучается в две фазы:

#### Фаза 1: Обучение с BATCH_SIZE_PHASE1=16. Меньшие батчи могут способствовать более быстрой сходимости в начале обучения.

#### Фаза 2: Продолжение обучения с BATCH_SIZE_PHASE2=32. Большие батчи могут обеспечить более стабильный градиент и лучшее обобщение на поздних этапах.

#### Колбэки (Callbacks): Для обеих фаз обучения используются одинаковые колбэки:

#### ModelCheckpoint: Сохраняет лучшую версию модели на основе метрики val_f1_m (F1 на валидационной выборке). save_best_only=True гарантирует сохранение только наилучшей модели.

#### EarlyStopping: Останавливает обучение, если val_f1_m не улучшается в течение PATIENCE_EARLY_STOPPING=25 эпох. restore_best_weights=True загружает веса лучшей эпохи после остановки.

#### ReduceLROnPlateau: Уменьшает скорость обучения, если val_f1_m не улучшается в течение PATIENCE_REDUCE_LR=6 эпох. Это помогает модели выйти из локальных минимумов.

#### Обучение модели: model.fit() запускает процесс обучения, используя созданные генераторы данных и колбэки.

#### Сохранение моделей: После завершения обеих фаз обучения (или ранней остановки), лучшая версия каждой обученной модели сохраняется на диск в формате h5.keras или keras в папку ensemble_models_custom. Пути и метаданные моделей записываются для последующего использования в ансамбле.

#### Завершение эксперимента Comet ML: experiment.end() закрывает текущий эксперимент в Comet ML, сохраняя все логи.

In [ ]:
input_shape = X_batch.shape[1:] 

trained_model_paths = [] 
model_metadata = []      

total_runs = len(MODEL_TYPES_FOR_ENSEMBLE) * NUM_RUNS_PER_MODEL_TYPE
run_counter = 0

for model_type in MODEL_TYPES_FOR_ENSEMBLE:
    for i in range(NUM_RUNS_PER_MODEL_TYPE):
        run_counter += 1
        print(f"\n--- Обучение модели {model_type} (Запуск {i+1}/{NUM_RUNS_PER_MODEL_TYPE}) ---")

        current_seed = BASE_RANDOM_SEED + run_counter * 10
        tf.random.set_seed(current_seed)
        np.random.seed(current_seed)
        os.environ['PYTHONHASHSEED'] = str(current_seed)

        if 'experiment' in globals() and isinstance(globals()['experiment'], comet_ml.Experiment):
            if not globals()['experiment'].ended:
                globals()['experiment'].end()
        experiment = comet_ml.Experiment(
            api_key=COMET_API_KEY,
            project_name="Classification for Landslide Detection Ensamble", 
            auto_output_logging="simple"
        )
        experiment.flush()
        experiment_name = f"{model_type}_run_{i+1}_seed_{current_seed}"
        experiment.set_name(experiment_name)

        print(f"--- Фаза 1: Обучение с Batch Size {BATCH_SIZE_PHASE1} на {EPOCHS_PHASE1} эпох ---")
        train_gen_phase1 = LandslideDataGenerator(
            train_df_split, folder_path, batch_size=BATCH_SIZE_PHASE1, augment=True, shuffle=True)
        val_gen_phase1 = LandslideDataGenerator(
            val_df_split, folder_path, batch_size=BATCH_SIZE_PHASE1, augment=False, shuffle=False)

        model = create_compiled_model(model_type, input_shape)

        callbacks_phase1 = [
            ModelCheckpoint(
                f"best_model{experiment_name}.h5.keras",
                monitor='val_f1_m',
                mode='max',
                save_best_only=True,
                verbose=3
            ),
            EarlyStopping(
                monitor='val_f1_m', 
                mode='max',
                patience=PATIENCE_EARLY_STOPPING,
                verbose=2,
                restore_best_weights=True 
            ),
            ReduceLROnPlateau(
                monitor='val_f1_m',
                mode='max',
                factor=0.4,
                patience=PATIENCE_REDUCE_LR,
                min_lr=MIN_LR,
                verbose=2
            ),
        ]

        history_phase1 = model.fit(
            train_gen_phase1,
            epochs=EPOCHS_PHASE1,
            validation_data=val_gen_phase1,
            callbacks=callbacks_phase1,
        )

        print(f"\n--- Фаза 2: Продолжение обучения с Batch Size {BATCH_SIZE_PHASE2} на макс. {EPOCHS_PHASE2_MAX} эпох ---")
        train_gen_phase2 = LandslideDataGenerator(
            train_df_split, folder_path, batch_size=BATCH_SIZE_PHASE2, augment=True, shuffle=True)
        val_gen_phase2 = LandslideDataGenerator(
            val_df_split, folder_path, batch_size=BATCH_SIZE_PHASE2, augment=False, shuffle=False)

        callbacks_phase2 = [
            ModelCheckpoint(
                f"best_model{experiment_name}.h5.keras",
                monitor='val_f1_m',
                mode='max',
                save_best_only=True,
                verbose=3
            ),
            EarlyStopping(
                monitor='val_f1_m',
                mode='max',
                patience=PATIENCE_EARLY_STOPPING,
                restore_best_weights=True,
                verbose=2
            ),
            ReduceLROnPlateau(
                monitor='val_f1_m',
                mode='max',
                factor=0.4,
                patience=PATIENCE_REDUCE_LR,
                min_lr=MIN_LR,
                verbose=2
            ),
        ]

        history_phase2 = model.fit(
            train_gen_phase2,
            epochs=EPOCHS_PHASE2_MAX,
            validation_data=val_gen_phase2,
            callbacks=callbacks_phase2,
        )

        model_save_dir = 'ensemble_models_custom'
        os.makedirs(model_save_dir, exist_ok=True)
        model_save_path = os.path.join(model_save_dir, f'{experiment_name}.keras')
        model.save(model_save_path, save_format='keras')
        print(f"Модель {model_type} (Запуск {i+1}) сохранена в {model_save_path}")

        trained_model_paths.append(model_save_path)
        model_metadata.append({'type': model_type, 'seed': current_seed, 'path': model_save_path})

        experiment.end()

print("\n--- Обучение всех моделей ансамбля завершено ---")

### График для F1 на валидационной выборке для всех 6 моделей из ансамбля ( по 2 EfficientnetB4, ResNet50, DenseNet169)
![ensamble.png](https://github.com/MakarRybkin/Slide_and_seek_classification/raw/master/images_for_report/ensamble.png)


### Получение предсказаний для каждой обученной модели из ансамбля и усреднение ответов

In [ ]:
from tensorflow.keras.models import  load_model
all_predictions = []

print("\n--- Получение предсказаний каждой обученной моделью ---")
for model_path in trained_model_paths:
    print(f"Загрузка модели: {model_path}")

    model = load_model(model_path,compile=False)

    preds = model.predict(val_gen, verbose=1)
    all_predictions.append(preds)

    tf.keras.backend.clear_session()

all_predictions = np.array(all_predictions)

ensemble_probabilities = np.mean(all_predictions, axis=0) 


### Определение генератора данных для тестовой выборки (LandslideTestGenerator). Также как и LandslideDataGenerator загружает данные батчами

In [ ]:
class LandslideTestGenerator(Sequence):
    def __init__(self, df, data_path, batch_size=32):
        self.df = df.reset_index(drop=True)
        self.data_path = data_path
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))

    def __getitem__(self, index):
        batch_df = self.df.iloc[index * self.batch_size:(index + 1) * self.batch_size]
        batch_X = []

        for _, row in batch_df.iterrows():
            image_id = row['ID']
            image = load_and_normalize_npy_image(image_id, self.data_path)
            batch_X.append(image)

        return np.array(batch_X)


### Выполнение предсказаний на тестовой выборке и создание файла для отправки

In [ ]:
test_df = pd.read_csv(test_csv_path)

test_gen = LandslideTestGenerator(test_df, test_data_path, batch_size=32)

all_test_predictions = []

print("Making predictions with each trained model on the Test set:")
for model_path in trained_model_paths:
    print(f"Loading model for test predictions: {model_path}")
    
    model = load_model(model_path, compile=False)

    preds_on_test = model.predict(test_gen, verbose=1)
    all_test_predictions.append(preds_on_test)
    
    tf.keras.backend.clear_session()

all_test_predictions = np.array(all_test_predictions)
ensemble_test_probabilities = np.mean(all_test_predictions, axis=0)

y_test_pred = (ensemble_test_probabilities > 0.5).astype(int)

unique, counts = np.unique(y_test_pred, return_counts=True)
prediction_counts = dict(zip(unique, counts))
print("Prediction counts:", prediction_counts)

submission_df = pd.DataFrame({
    'ID': test_df['ID'],
    'label': y_test_pred.flatten()
})
submission_df.to_csv(f'Submission_File{experiment_name}.csv', index=False)
print(f"Sample submission file created as 'Submission_File{experiment_name}.csv'.")


## Заключение
### Мы успешно разработали и обучили систему обнаружения оползней, используя комплексный подход к мультиспектральным спутниковым данным Sentinel-1 (SAR) и Sentinel-2 (оптические). Для достижения оптимальных результатов было проведено более 50 экспериментов.
### Ключевые методы и достижения:

#### Мультимодальная интеграция данных: Эффективно объединили и обработали данные с разных сенсоров. Это позволило модели извлекать комплементарную информацию, что особенно ценно в условиях облачности, когда оптические данные недоступны.

#### Борьба с дисбалансом классов: Оползни — редкие события, создающие сильный дисбаланс классов. Мы применили Focal Loss для фокусировки на миноритарном и "сложном" классах. Оценка производительности основывалась на F1-мере, Precision и Recall, более релевантных для несбалансированных данных, чем простая точность.

#### Аугментация данных: Использование разнообразных техник аугментации (повороты, сдвиги, масштабирование, отражения, изменение яркости/контраста, шум) значительно расширило обучающий набор, повысило устойчивость модели и эффективно предотвратило переобучение.

#### Ансамблевое обучение и стратегии оптимизации: Обучили ансамбль из нескольких моделей, включая EfficientNetB5, DenseNet169 и ResNet50. Каждая модель проходила двухфазное обучение с уменьшением размера батча на ранних этапах для ускорения сходимости и увеличением на поздних для стабильности. Использовали Early Stopping для предотвращения переобучения и ReduceLROnPlateau для адаптивного управления скоростью обучения. Усреднение предсказаний ансамбля дало более стабильные и точные итоговые результаты.

#### Отслеживание экспериментов: Интеграция с Comet ML обеспечила прозрачное отслеживание, сравнение и воспроизводимость всех экспериментов.

### Итоговый результат на текущий момент (20 дней до завершения соревнования):

### Благодаря примененным стратегиям, нам удалось достичь F1 = 0.8486. Этот результат позволил занять 47-е место в общем рейтинге соревнования.
![current_place.png](https://github.com/MakarRybkin/Slide_and_seek_classification/raw/master/images_for_report/bcurrent_place.png)

### Дальнейшие шаги
### Я уже активно экспериментирую с гибридными архитектурами CNN + Трансформеры. Цель — объединить сильные стороны CNN (локальное извлечение признаков) и Трансформеров (моделирование глобального контекста) для дальнейшего повышения точности обнаружения оползней.

